In [2]:
import numpy as np
import pandas as pd

import nltk
nltk.download("opinion_lexicon")
from nltk.corpus import opinion_lexicon

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation



[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\iamen\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [4]:
tweets = pd.read_csv(r"Sentiment Analysis Dataset.csv")
tweets = tweets[["SentimentText", "Sentiment"]]
tweets

,SentimentText,Sentiment
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,1
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0
...,...,...
1578591,Zzzzzz.... Finally! Night tweeters!,1
1578592,"Zzzzzzz, sleep well people",1
1578593,ZzzZzZzzzZ... wait no I have homework.,0
1578594,"ZzZzzzZZZZzzz meh, what am I doing up again?",0


In [5]:
lemma = WordNetLemmatizer()
stp_wds = set(stopwords.words("english")+list(punctuation))
stp_wds

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [7]:
def get_lem(text):
    text = text.lower()
    words = word_tokenize(text)
    words_stop = []
    for word in words:
        if word in stp_wds: continue
        words_stop.append(word)
    return [lemma.lemmatize(word) for word in words_stop]

def pos_neg(lemm_list):
    pos_words = []
    neg_words = []
    for word in lemm_list:
        if word in opinion_lexicon.negative(): neg_words.append(word)
        if word in opinion_lexicon.positive(): pos_words.append(word)
    return (pos_words, neg_words)

In [8]:
tweets[["SentimentText"]]

,SentimentText
0,is so sad for my APL frie...
1,I missed the New Moon trail...
2,omg its already 7:30 :O
3,.. Omgaga. Im sooo im gunna CRy. I'...
4,i think mi bf is cheating on me!!! ...
...,...
1578591,Zzzzzz.... Finally! Night tweeters!
1578592,"Zzzzzzz, sleep well people"
1578593,ZzzZzZzzzZ... wait no I have homework.
1578594,"ZzZzzzZZZZzzz meh, what am I doing up again?"


In [ ]:
senti_pred = []
for i in list(tweets["SentimentText"]):

    this = pos_neg(get_lem(i))
    no_pos, no_neg = len(this[0]), len(this[1])
    if no_pos + no_neg > 0: 
        if((no_pos - no_neg)/(no_pos + no_neg)) > 0: senti_pred.append(1)
        else: senti_pred.append(0)
    else: 
        senti_pred.append(1)

In [ ]:
tweets["senti_pred"] = senti_pred

In [ ]:
tweets["senti_pred"]